In [54]:
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import json 
!pip install geopy
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!pip install folium
import folium

print('Libraries imported.')

Libraries imported.


In [0]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

# Link Submission 1

#### Scrape wikipedia page for Canada postal codes


In [0]:
df = pd.read_html(url)[0]

In [5]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### Find row where Borough is available but Neighbourhood is "Not assigned".  Once found, I will replace "Not assigned" with Borough name.

In [6]:
df.loc[(df['Neighbourhood'] == "Not assigned") & (df['Borough'] != "Not assigned")]

,Postcode,Borough,Neighbourhood
9,M9A,Queen's Park,Not assigned


In [0]:
df.at[9,'Neighbourhood'] = df.at[9,'Borough']

#### Find rows where Borough and Neighbourhood are both "Not assigned"

In [8]:
rows_to_drop = df.loc[(df['Neighbourhood'] == "Not assigned") & (df['Borough'] == "Not assigned")]
rows_to_drop.index.array

<PandasArray>
[  0,   1,   8,  12,  19,  20,  29,  35,  36,  44,  45,  49,  50,  51,  53,
  54,  58,  59,  60,  72,  73,  74,  87,  88,  89, 103, 104, 105, 119, 120,
 135, 136, 147, 148, 154, 160, 161, 166, 174, 180, 181, 187, 188, 189, 193,
 194, 200, 201, 202, 203, 208, 209, 222, 223, 236, 237, 240, 241, 246, 247,
 252, 253, 257, 258, 259, 260, 262, 263, 273, 274, 275, 276, 277, 278, 279,
 280, 286]
Length: 77, dtype: int64

#### Drop the rows

In [10]:
cleaned_df = df.drop(rows_to_drop.index.array)
cleaned_df.reset_index(drop=True)
cleaned_df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


#### Group by Postcode and Borough and aggregate Neighbourhoods belonging to the same borough and separate with ","

In [0]:
final_df = cleaned_df.groupby(['Postcode', 'Borough']).agg(Neighbourhood=('Neighbourhood',','.join)).reset_index()

#### Display final DataFrame and shape

In [13]:
final_df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [15]:
final_df.shape

(103, 3)

# Link Submission 2

#### Import geocoder and test with location

In [17]:
!pip install geocoder
import geocoder # import geocoder
print('geocoder imported!')

     |████████████████████████████████| 102kB 2.3MB/s 
geocoder imported!


In [207]:
# Google had reliability issues therefore geocodefarm was used
g = geocoder.geocodefarm('M1B, Toronto, Ontario')
g.json


{'accuracy': 'EXACT_MATCH',
 'address': 'M1B, ON, Canada',
 'bbox': {'northeast': [43.7854690551717, -79.2443237304129],
  'southwest': [43.8348388671718, -79.1338729851296]},
 'city': 'Toronto',
 'confidence': 4,
 'country': 'Canada',
 'county': 'Toronto',
 'elevation': 'UNAVAILABLE',
 'lat': 43.8101539611717,
 'lng': -79.1946029663129,
 'ok': True,
 'postal': 'M1B',
 'raw': {'ADDRESS': {'admin_1': 'ON',
   'admin_2': 'Toronto',
   'country': 'Canada',
   'locality': 'Toronto',
   'postal_code': 'M1B'},
  'BOUNDARIES': {'northeast_latitude': '43.7854690551717',
   'northeast_longitude': '-79.2443237304129',
   'southwest_latitude': '43.8348388671718',
   'southwest_longitude': '-79.1338729851296'},
  'COORDINATES': {'latitude': '43.8101539611717',
   'longitude': '-79.1946029663129'},
  'LOCATION_DETAILS': {'elevation': 'UNAVAILABLE',
   'timezone_long': 'UNAVAILABLE',
   'timezone_short': 'America/Toronto'},
  'accuracy': 'EXACT_MATCH',
  'formatted_address': 'M1B, ON, Canada'},
 'st

In [19]:
# create a numpy array of postcodes in order to iterate over
codes_arr = final_df['Postcode'].to_numpy()
codes_arr

array(['M1B', 'M1C', 'M1E', 'M1G', 'M1H', 'M1J', 'M1K', 'M1L', 'M1M',
       'M1N', 'M1P', 'M1R', 'M1S', 'M1T', 'M1V', 'M1W', 'M1X', 'M2H',
       'M2J', 'M2K', 'M2L', 'M2M', 'M2N', 'M2P', 'M2R', 'M3A', 'M3B',
       'M3C', 'M3H', 'M3J', 'M3K', 'M3L', 'M3M', 'M3N', 'M4A', 'M4B',
       'M4C', 'M4E', 'M4G', 'M4H', 'M4J', 'M4K', 'M4L', 'M4M', 'M4N',
       'M4P', 'M4R', 'M4S', 'M4T', 'M4V', 'M4W', 'M4X', 'M4Y', 'M5A',
       'M5B', 'M5C', 'M5E', 'M5G', 'M5H', 'M5J', 'M5K', 'M5L', 'M5M',
       'M5N', 'M5P', 'M5R', 'M5S', 'M5T', 'M5V', 'M5W', 'M5X', 'M6A',
       'M6B', 'M6C', 'M6E', 'M6G', 'M6H', 'M6J', 'M6K', 'M6L', 'M6M',
       'M6N', 'M6P', 'M6R', 'M6S', 'M7A', 'M7R', 'M7Y', 'M8V', 'M8W',
       'M8X', 'M8Y', 'M8Z', 'M9A', 'M9B', 'M9C', 'M9L', 'M9M', 'M9N',
       'M9P', 'M9R', 'M9V', 'M9W'], dtype=object)

#### Iterate over postcodes array and extract latitude and longitude values

In [214]:
coordinates = []
counter = 0
for code in codes_arr:
  lat_lng_coords = None

# loop until you get the coordinates
  while(lat_lng_coords is None):
    g = geocoder.geocodefarm('{}, Toronto, Ontario'.format(code))
    lat_lng_coords = g.latlng

  latitude = lat_lng_coords[0]
  longitude = lat_lng_coords[1]
  latlng = (code, latitude, longitude)
  coordinates.append(latlng)
  counter += 1
  print(code,latitude,longitude)

print("Process Completed with {} calls to Foursquare".format(counter))

M1B 43.8101539611717 -79.1946029663129
M1C 43.7846717834717 -79.1589584350129
M1E 43.7662887573717 -79.1728897094129
M1G 43.7682876586717 -79.2141113281129
M1H 43.7691802978717 -79.2387695311298
M1J 43.7439384460716 -79.2313537597129
M1K 43.7258033752716 -79.2628479003129
M1L 43.7166557312716 -79.2865371701299
M1M 43.7235755920716 -79.2344512939129
M1N 43.6978836059715 -79.2587585449129
M1P 43.7598838806716 -79.2683334350129
M1R 43.7513008117716 -79.2998123168129
M1S 43.7977066040717 -79.2677078247129
M1T 43.7849998474717 -79.2993545532129
M1V 43.8178100585717 -79.2865142822129
M1W 43.8015632629717 -79.3188705444129
M1X 43.8344345092718 -79.2189102172129
M2H 43.8029403686717 -79.356063842713
M2J 43.7811393737717 -79.348510742113
M2K 43.7794151306717 -79.37744140613
M2L 43.7575950622716 -79.380737304613
M2M 43.7921752929717 -79.413848876913
M2N 43.7682800292717 -79.407455444313
M2P 43.7491798400716 -79.39917755113
M2R 43.7771759037172 -79.442657470713
M3A 43.7512550371682 -79.3298950195

In [21]:
print(coordinates)

[('M1B', 43.8101539611717, -79.1946029663129), ('M1C', 43.7846717834717, -79.1589584350129), ('M1E', 43.7662887573717, -79.1728897094129), ('M1G', 43.7682876586717, -79.2141113281129), ('M1H', 43.7691802978717, -79.2387695311298), ('M1J', 43.7439384460716, -79.2313537597129), ('M1K', 43.7258033752716, -79.2628479003129), ('M1L', 43.7166557312716, -79.2865371701299), ('M1M', 43.7235755920716, -79.2344512939129), ('M1N', 43.6978836059715, -79.2587585449129), ('M1P', 43.7598838806716, -79.2683334350129), ('M1R', 43.7513008117716, -79.2998123168129), ('M1S', 43.7977066040717, -79.2677078247129), ('M1T', 43.7849998474717, -79.2993545532129), ('M1V', 43.8178100585717, -79.2865142822129), ('M1W', 43.8015632629717, -79.3188705444129), ('M1X', 43.8344345092718, -79.2189102172129), ('M2H', 43.8029403686717, -79.356063842713), ('M2J', 43.7811393737717, -79.348510742113), ('M2K', 43.7794151306717, -79.37744140613), ('M2L', 43.7575950622716, -79.380737304613), ('M2M', 43.7921752929717, -79.41384887

#### Create a DataFrame of postcode, latitude, and longitude

In [0]:
coord_df = pd.DataFrame(data=coordinates, columns=['Postcode', 'Latitude', 'Longitude'])


In [208]:
coord_df.head()

,Postcode,Latitude,Longitude
0,M1B,43.810154,-79.194603
1,M1C,43.784672,-79.158958
2,M1E,43.766289,-79.172890
3,M1G,43.768288,-79.214111
4,M1H,43.769180,-79.238770


In [0]:
# export to csv
# coord_df.to_csv('coordinates.csv', index=False)

In [0]:
# create dataframe from saved csv
# coordinates_df = pd.read_csv('coordinates.csv')
# coordinates_df.head()

#### Merge dataframes and display

In [0]:
geo_df = final_df.merge(coord_df, on='Postcode')

In [29]:
geo_df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.810154,-79.194603
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784672,-79.158958
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.766289,-79.172890
3,M1G,Scarborough,Woburn,43.768288,-79.214111
4,M1H,Scarborough,Cedarbrae,43.769180,-79.238770


# Link Submission 3

#### Create an initial folium map with Toronto coordinates to start

In [30]:
toronto_map = folium.Map(location=[43.7184038, -79.5181407], zoom_start=11)

for lat, lng, postcode, borough, neighbourhood in zip(geo_df['Latitude'], geo_df['Longitude'], geo_df['Postcode'], geo_df['Borough'], geo_df['Neighbourhood']):
    label = 'Postcode: {} \n Neighbourhood: {} \n Borough: {}'.format(postcode, neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='purple',
        fill=True,
        fill_color='purple',
        fill_opacity=0.2,
        parse_html=False).add_to(toronto_map)  

toronto_map

In [31]:

CLIENT_ID = 'BQ0CRPJBBWUOZK2EVOGPAX3UJ3YHPGDRR3CYXOAQBJ5PJ1XX' # your Foursquare ID
CLIENT_SECRET = 'VXTX3QW2ZSEV135W5C4TFPJD0JYWEQFFB5BZR5PCX3ASYD5Q' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: BQ0CRPJBBWUOZK2EVOGPAX3UJ3YHPGDRR3CYXOAQBJ5PJ1XX
CLIENT_SECRET:VXTX3QW2ZSEV135W5C4TFPJD0JYWEQFFB5BZR5PCX3ASYD5Q


In [0]:
# geo_df.loc[40, 'Neighbourhood']

In [0]:
# neighbourhood_latitude = geo_df.loc[0, 'Latitude']
# neighbourhood_longitude = geo_df.loc[0, 'Longitude']

# neighbourhood_name = geo_df.loc[0, 'Neighbourhood']

# print('{} Latitude: {}, Longitude: {}'.format(neighbourhood_name,neighbourhood_latitude,neighbourhood_longitude))

In [0]:
# Top 100 venues in neighborhood
# radius = 1200
# LIMIT = 100
# url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighbourhood_latitude, neighbourhood_longitude, VERSION, radius, LIMIT)
# url

In [0]:
# results = requests.get(url).json()
# results

#### Create a function to extract category names of each venue

In [0]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#### Traverse json results to extract items, create, and display new dataframe

In [122]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Images Salon & Spa,Spa,43.802283,-79.198565
1,Canadiana exhibit,Zoo Exhibit,43.817962,-79.193374
2,Lion Exhibit,Zoo Exhibit,43.819228,-79.186977
3,Wendy's,Fast Food Restaurant,43.807448,-79.199056
4,Wendy's,Fast Food Restaurant,43.802008,-79.198080


In [123]:
print('Foursquare has returned {} venues.'.format(nearby_venues.shape[0]))

Foursquare has returned 25 venues.


#### Neighborhood Exploration

#### Create a function which will use the Foursquare API to extract venues from coordinates supplied.

In [0]:
def getNearbyVenues(names, latitudes, longitudes, radius=1200): # I am increasing the radius here in order to avoid NaN values when clustering
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Use function on geo_df DataFrame

In [125]:
toronto_venues = getNearbyVenues(names=geo_df['Neighbourhood'],
                                   latitudes=geo_df['Latitude'],
                                   longitudes=geo_df['Longitude']
                                  )
print('-------------------')
print('Process Complete')

Rouge,Malvern
Highland Creek,Rouge Hill,Port Union
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park,Ionview,Kennedy Park
Clairlea,Golden Mile,Oakridge
Cliffcrest,Cliffside,Scarborough Village West
Birch Cliff,Cliffside West
Dorset Park,Scarborough Town Centre,Wexford Heights
Maryvale,Wexford
Agincourt
Clarks Corners,Sullivan,Tam O'Shanter
Agincourt North,L'Amoreaux East,Milliken,Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview,Henry Farm,Oriole
Bayview Village
Silver Hills,York Mills
Newtonbrook,Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park,Don Mills South
Bathurst Manor,Downsview North,Wilson Heights
Northwood Park,York University
CFB Toronto,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens,Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West,Riverdale
The Beaches West,Indi

In [126]:
print(toronto_venues.shape)
toronto_venues.head()

(5886, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge,Malvern",43.810154,-79.194603,Images Salon & Spa,43.802283,-79.198565,Spa
1,"Rouge,Malvern",43.810154,-79.194603,Canadiana exhibit,43.817962,-79.193374,Zoo Exhibit
2,"Rouge,Malvern",43.810154,-79.194603,Lion Exhibit,43.819228,-79.186977,Zoo Exhibit
3,"Rouge,Malvern",43.810154,-79.194603,Wendy's,43.807448,-79.199056,Fast Food Restaurant
4,"Rouge,Malvern",43.810154,-79.194603,Wendy's,43.802008,-79.198080,Fast Food Restaurant


In [128]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Agincourt,47,47,47,47,47,47
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",50,50,50,50,50,50
"Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown",19,19,19,19,19,19
"Alderwood,Long Branch",37,37,37,37,37,37
"Bathurst Manor,Downsview North,Wilson Heights",33,33,33,33,33,33
Bayview Village,30,30,30,30,30,30
"Bedford Park,Lawrence Manor East",54,54,54,54,54,54
Berczy Park,100,100,100,100,100,100


In [129]:
print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 346 unique categories.


#### Analyze all neighborhoods in dataset

In [131]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood']

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
# print(manhattan_onehot.columns[-1])
# print(list(manhattan_onehot.columns[:-1]))
# print([manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1]))
toronto_onehot.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Daycare,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Elementary School,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Hungarian Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Leather Goods Store,Light Rail Station,Lighthouse,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoor Supply Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Photography Lab,Pide Place,Pie Shop,Pier,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Science Museum,Sea

In [132]:
toronto_onehot.shape

(5886, 347)

In [133]:
toronto_group = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_group.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Daycare,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Elementary School,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Hungarian Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Leather Goods Store,Light Rail Station,Lighthouse,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoor Supply Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Photography Lab,Pide Place,Pie Shop,Pier,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Science Museum,Sea

In [134]:
toronto_group.shape

(102, 347)

#### Loop through neighborhoods in grouped dataframe to extract venue and frequency of appearance

In [135]:
top_venue_ct = 5

for neigh in toronto_group['Neighbourhood']:
    print("---- "+neigh+" ----")
    temp = toronto_group[toronto_group['Neighbourhood'] == neigh].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(top_venue_ct))
    print('\n')

---- Adelaide,King,Richmond ----
            venue  freq
0     Coffee Shop  0.08
1            Café  0.06
2           Hotel  0.05
3         Theater  0.04
4  Clothing Store  0.03


---- Agincourt ----
                venue  freq
0  Chinese Restaurant  0.13
1       Shopping Mall  0.06
2                Park  0.04
3         Pizza Place  0.04
4      Sandwich Place  0.04


---- Agincourt North,L'Amoreaux East,Milliken,Steeles East ----
                venue  freq
0  Chinese Restaurant  0.20
1              Bakery  0.08
2           BBQ Joint  0.06
3         Pizza Place  0.06
4                Park  0.06


---- Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown ----
                 venue  freq
0        Grocery Store  0.16
1          Coffee Shop  0.11
2                 Park  0.11
3          Pizza Place  0.11
4  Fried Chicken Joint  0.05


---- Alderwood,Long Branch ----
            venue  freq
0     Pizza Place  0.08
1        Pharmacy  0.08
2   

#### Create a function to return the most common venues based on a determined count

In [0]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Create a dataframe of neighborboods and 10 most common venues using function above

In [201]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_group['Neighbourhood']

for ind in np.arange(toronto_group.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_group.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Hotel,Theater,Japanese Restaurant,Clothing Store,Pizza Place,Breakfast Spot,Movie Theater,Beer Bar
1,Agincourt,Chinese Restaurant,Shopping Mall,Sandwich Place,Park,Pizza Place,Fast Food Restaurant,Restaurant,Coffee Shop,Bakery,Bank
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Chinese Restaurant,Bakery,Pizza Place,Park,BBQ Joint,Grocery Store,Pharmacy,Korean Restaurant,Noodle House,Steakhouse
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Park,Pizza Place,Coffee Shop,Sandwich Place,Pharmacy,Gym Pool,Beer Store,Discount Store,Hardware Store
4,"Alderwood,Long Branch",Coffee Shop,Pharmacy,Pizza Place,Discount Store,Convenience Store,Café,Park,Light Rail Station,Restaurant,Burger Joint


In [0]:
# neighbourhoods_venues_sorted
# geo_df.head(50)

## Cluster Neighborhoods

In [203]:
# set number of clusters
kclusters = 5

toronto_group_cluster = toronto_group.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_group_cluster)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 3, 3, 3, 3, 3, 3, 3, 2, 3], dtype=int32)

In [204]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = geo_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",43.810154,-79.194603,2,Zoo Exhibit,Fast Food Restaurant,Trail,Coffee Shop,Chinese Restaurant,Paper / Office Supplies Store,Auto Workshop,Martial Arts Dojo,Bus Station,Other Great Outdoors
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784672,-79.158958,1,Breakfast Spot,Gym / Fitness Center,Park,Playground,Burger Joint,Italian Restaurant,Farmers Market,Elementary School,Empanada Restaurant,Ethiopian Restaurant
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.766289,-79.172890,3,Pizza Place,Park,Breakfast Spot,Convenience Store,Coffee Shop,Bar,Gym / Fitness Center,Gym,Grocery Store,Juice Bar
3,M1G,Scarborough,Woburn,43.768288,-79.214111,3,Indian Restaurant,Pizza Place,Department Store,Mobile Phone Shop,Sandwich Place,Chinese Restaurant,Juice Bar,Thrift / Vintage Store,Discount Store,Park
4,M1H,Scarborough,Cedarbrae,43.769180,-79.238770,3,Coffee Shop,Indian Restaurant,Pharmacy,Bakery,Gas Station,Pizza Place,Fried Chicken Joint,Board Shop,Caribbean Restaurant,Music Store


# Display final map with clusters

In [206]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters